# Import Required Libraries
This cell imports the necessary Python libraries for the analysis. It includes the custom `functions_main` library, `numpy` and `numpy.random` for numerical calculations, `matplotlib.pyplot` for data visualization.

In [12]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"   # mac/Accelerate  
from functions_main import *   
import numpy as np
import cvxpy as cp
import numpy.random as rgt
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.ticker import MaxNLocator 
from joblib import Parallel, delayed
from tqdm.notebook import tqdm  
from matplotlib.lines import Line2D

# Model for High-Dimensional Data 
We simulate high-dimensional data based on the model:


Y =  X β +  ε  


where X ~ N(0, Sigma) with Sigma_{ij} = 0.1^{|i-j|}; the first s^* elements of β are set to either 1 or -1, and the remaining elements are zero. 


##   Inputs required

n/p: sample size/dimension  

Gaussian_error: (1)True, ε ~ N(0,1);   (2)False,   ε ~ t_{2.25} 

## Functions for initialization

In [4]:
####################### 1. DP support recovery ########################
def dp_top_s_laplace_peeling(u , s , eps , Delta , keep_order  = True, rng=None): 
    rng = np.random.default_rng() if rng is None else rng
    p = u.shape[0]
    s = int(min(max(s, 0), p))
    if s == 0: return np.array([], dtype=int)
    eps_each = eps / s
    scale    = Delta / eps_each   # = (s * Delta) / eps
    chosen = []
    mask   = np.ones(p, dtype=bool)
    for _ in range(s):
        noise = rng.laplace(0.0, scale, size=p)
        scores = np.where(mask, u + noise, -np.inf)
        j = int(np.argmax(scores))
        chosen.append(j)
        mask[j] = False
    idx = np.array(chosen, dtype=int)
    if keep_order:
        idx.sort()                
    return idx

####################### 2. DP estimation of standard deviation ########################
def St_DP(Y, epsilon):

    """
    (ε,0)-DP estimator of the standard deviation based on clipping and the Laplace mechanism.

    Parameters
    ----------
    Y : n by 1 numpy array of response variables. 
    epsilon :  privacy parameter.
    """

    n =  Y.size
    gamma = np.log(n)
    y =  np.clip(np.asarray(Y, dtype=float), -gamma, gamma)
    m1 = y.mean()
    m2 = np.mean(y**2)  
    Delta1 = 2*gamma / n
    Delta2 = (gamma**2) / n

    epsilon1 = epsilon2 = epsilon /2    # Split the privacy budget equally between the mean and second moment
    b1 = Delta1 / float(epsilon1)
    b2 = Delta2 / float(epsilon2)
    noise1 =  rgt.laplace(0.0, b1)
    noise2 =  rgt.laplace(0.0, b2) 
         
    m1_dp = m1 + noise1
    m2_dp = m2 + noise2
    var_dp = float(m2_dp - m1_dp**2)
    if var_dp > 0:
        st_dp = np.sqrt(var_dp)
    else:
        st_dp = 2.0
    return st_dp

#####################  3. huber & ridge initialization #####################  
def huber_ridge_priv(X, Y, tau, epsilon, delta): 

    '''
    Huber + ridge regression & output perturbation
    
    Parameters
    ---------
    X : n by p0 numpy array of covariates; each row is an observation vector. (no intercept)
    Y : n by 1 numpy array of response variables. 
    tau : (DP) robust parameter for huber regression.
    epsilon,delta : privacy parameters.   
    '''

    n, p0 = X.shape
    gamma = np.sqrt(p0+1)/6 
    X_t = clipping_l2(X, gamma) 
 
    Z = np.hstack([np.ones((n, 1)), X_t])   # [1, X̃]
    beta = cp.Variable(p0 + 1) 

    resid = Y - Z  @ beta
    lam = 0.2 # Ridge regularization parameter
    obj = (1.0 / n) * cp.sum(cp.huber(resid, tau)) + 0.5 * lam * cp.sum_squares(  beta) 

    prob = cp.Problem(cp.Minimize(obj)) 
    prob.solve(solver="SCS", verbose=False, warm_start=True)

    b = np.asarray(beta.value).ravel() 
    B = np.sqrt(1 + gamma**2)
    noise_scale = 2*B*tau*((  ( 2 *np.log(1.25 /delta))**0.5 ))/(n*epsilon*lam)
    bDP = b + noise_scale * rgt.standard_normal(p0 + 1)
    return bDP  
 

## Main function

In [ ]:
def main_priv_est(n,p,Gaussian_error=True):
    # true beta
    rgt.seed(0) # set seed
    beta = np.zeros(p)  # initial beta  
    s_star = 10
    beta[:s_star] =  np.ones(s_star)*(2*rgt.binomial(1, 0.5, size=s_star)-1)  
    beta_norm = beta.dot(beta)**0.5 
    beta_slope = beta[1:]
    beta_slope_norm = beta_slope.dot(beta_slope)**0.5 
    
    #privacy parameters 
    epsilon = np.array([0.5,0.9 ])
    delta  = 10/(n**1.1)
    eps_s = epsilon/3
    eps_init = epsilon/3
    eps_main = epsilon/3  
    delta_init = delta/2
    delta_main = delta/2   
    
    repetitions = 300 # number of repetitions    
    rela_l2 = np.zeros([  repetitions])  # relative l2-error results of the non-private estimate 
    priv_rela_l2_1 = np.zeros([  repetitions]) # relative l2-error results of the private estimate under epsilon = 0.5
    priv_rela_l2_2  = np.zeros([  repetitions]) # relative l2-error results of the private estimate under epsilon = 0.9
    rela_l2_comp = np.zeros([  repetitions])  # for slope estimation comparison
    priv_rela_l2_comp = np.zeros([  repetitions])  # for slope estimation comparison 
    for m in range(repetitions):
        rgt.seed(m+10) # set seed
        
        ############# generate data ################ 
        # X 
        idx = np.arange(p-1)
        Sigma = 0.1 ** np.abs(idx[:, None] - idx[None, :])
        L = np.linalg.cholesky(Sigma)  # Sigma = L @ L.T  (L lower-triangular)
        Z = rgt.normal(size=(n, p-1))  # iid N(0,1)
        X = Z @ L.T                        # each row has Cov = Sigma
        # error
        if Gaussian_error: err = rgt.normal(0,1, n)
        else: err = rgt.standard_t(2.25, n) 
        # Y
        Y = beta[0] + X.dot(beta[1:]) +  err
        

        ############################################### 
        #-------------  Private (proposed)  ----------- 
        ############################################### 

        #----------- index selection ----------- 
        xi = np.sqrt(np.log(n)+np.log(p))  
        U = X * Y[:, None]
        absU = np.abs(U)
        weights = np.where(absU > 0.0, np.minimum(xi / absU, 1.0), 1.0)
        U_tilde = U * weights
        g = np.abs(U_tilde.mean(axis=0)) 
        # Global replace-one sensitivity for each score: Δ = 2C / n
        Delta = 2.0 * xi / n
        s = int(np.ceil(1.2 * s_star))
        dx_peel_1 = dp_top_s_laplace_peeling(g, s-1, eps_s[0], Delta, rng=rgt)
        dx_peel_2 = dp_top_s_laplace_peeling(g, s-1, eps_s[1], Delta, rng=rgt)

        #----------- initialization -----------
        support_org_1 = np.asarray(list(dx_peel_1), dtype=int)
        support_org_2 = np.asarray(list(dx_peel_2), dtype=int)
        X_low_1 = X[:,support_org_1]
        X_low_2 = X[:,support_org_2]
        epsilon_tau =  eps_init/4  # privacy budget for DP standard deviation estimation: 1/4
        epsilon_ridge =  3*eps_init/4 # privacy budget for DP huber+ridge estimation: 3/4
        tau_init_1 =   St_DP(Y, epsilon = epsilon_tau[0]) 
        tau_init_2 =   St_DP(Y, epsilon = epsilon_tau[1])   
        beta_0DP_1 =  huber_ridge_priv(X_low_1, Y, tau=tau_init_1, epsilon= epsilon_ridge[0], delta= delta_init) 
        beta_0DP_2 =  huber_ridge_priv(X_low_2, Y, tau=tau_init_2, epsilon= epsilon_ridge[1], delta= delta_init)
        beta0DP_1 = np.zeros(p)
        support_selected_1 = np.insert(support_org_1 + 1, 0, 0).astype(int) 
        beta0DP_1[support_selected_1] = beta_0DP_1 
        beta0DP_2 = np.zeros(p)
        support_selected_2 = np.insert(support_org_2 + 1, 0, 0).astype(int) 
        beta0DP_2[support_selected_2] = beta_0DP_2 
        
        #----------- estimation -----------
        tau_priv_1 = .04 * tau_init_1 * (n*epsilon[0]/(s*np.log(p)+np.log(n)))**0.5 # robustification parameter for  DP    
        tau_priv_2 = .04 * tau_init_2 * (n*epsilon[1]/(s*np.log(p)+np.log(n)))**0.5  
        lr_priv =  0.01  
        B_priv = 0.5*(np.log(p) + np.log(n))**0.5
        T_priv = int(np.ceil(2*np.log(n))) # number of iterations
        if T_priv*np.log(T_priv/delta_main) < 2.5*np.log(2/delta_main)*np.log((2*T_priv)/delta_main):
            epsilon_scale_priv = eps_main/T_priv
            delta_scale_priv = delta_main/T_priv
        else: 
            epsilon_scale_priv = eps_main/np.sqrt(2.5*T_priv*np.log(2/delta_main))
            delta_scale_priv = delta_main/ (2*T_priv)
        model_est = Huber(X , Y ,intercept=True)
        out_priv_1 = model_est.noisygd_highdim( s=s, lr=lr_priv, T=T_priv,  tau=tau_priv_1 , beta0=beta0DP_1,  B_high=B_priv,  epsilon_scale =epsilon_scale_priv[0],  delta_scale=delta_scale_priv )    
        priv_rela_l2_1[m]  = np.log((np.sum((out_priv_1['beta']  - beta )**2 ))**0.5  / beta_norm )
        priv_rela_l2_comp[m] = np.log((np.sum((out_priv_1['beta'][1:]  - beta_slope )**2 ))**0.5  / beta_slope_norm ) # epsilon = 0.5 for comparison with DP-ls
        out_priv_2 = model_est.noisygd_highdim( s=s, lr=lr_priv, T=T_priv,  tau=tau_priv_2 , beta0=beta0DP_2,  B_high=B_priv,  epsilon_scale =epsilon_scale_priv[1],  delta_scale=delta_scale_priv )    
        priv_rela_l2_2[m]  = np.log((np.sum((out_priv_2['beta']  - beta )**2 ))**0.5  / beta_norm )
            
        
        ############################################### 
        #----------------  Non-private  --------------- 
        # ###############################################  

        tau0 = np.sqrt(np.mean(Y**2)-(np.mean(Y)**2))
        tau_np_high = .1 *tau0  * (n/(s*np.log(p)+np.log(n)))**0.5  # robustification parameter for noiseless Gaussian DP 
        lr_np =  0.2  
        T_np = int(np.ceil(2*np.log(n))) # number of iterations 
        model_est = Huber(X , Y ,intercept=True)
        out_np = model_est.gd_highdim(s=s, lr=lr_np, T=T_np,  tau=tau_np_high, beta0=np.array([]),  standardize= False) 
        rela_l2[m]  =  np.log((np.sum((out_np['beta']   - beta  )**2))**0.5  / beta_norm)
        rela_l2_comp[m] = np.log((np.sum((out_np['beta'][1:]  - beta_slope )**2 ))**0.5  / beta_slope_norm ) # for comparison with DP-ls

 
            
    return np.array([priv_rela_l2_1,priv_rela_l2_2,rela_l2,priv_rela_l2_comp,rela_l2_comp ])

In [ ]:
#test
re_high_test = main_priv_est(5000,5000,Gaussian_error=True)

## Main function  adapted for parallel computing

In [ ]:
def main_priv_est_parallel(m): 
    n_set = np.array([5000,10000,15000  ])
    p_set = np.array([5000 , 10000      ])  
    TF_set = np.array([True ,False])
    results = []
    for p in p_set: 
        for Gaussian_error in TF_set: 
            for n in n_set:
                # true beta
                rgt.seed(0) # set seed
                beta = np.zeros(p)  # initial beta  
                s_star = 10
                beta[:s_star] =  np.ones(s_star)*(2*rgt.binomial(1, 0.5, size=s_star)-1)  
                beta_norm = beta.dot(beta)**0.5 
                beta_slope = beta[1:]
                beta_slope_norm = beta_slope.dot(beta_slope)**0.5 
                
                #privacy parameters 
                epsilon = np.array([0.5,0.9 ])
                delta  = 10/(n**1.1)
                eps_s = epsilon/3
                eps_init = epsilon/3
                eps_main = epsilon/3  
                delta_init = delta/2
                delta_main = delta/2  

                rgt.seed(m+10) # set seed
                
                ############# generate data ################ 
                # X 
                idx = np.arange(p-1)
                Sigma = 0.1 ** np.abs(idx[:, None] - idx[None, :])
                L = np.linalg.cholesky(Sigma)  # Sigma = L @ L.T  (L lower-triangular)
                Z = rgt.normal(size=(n, p-1))  # iid N(0,1)
                X = Z @ L.T                        # each row has Cov = Sigma
                # error
                if Gaussian_error: err = rgt.normal(0,1, n)
                else: err = rgt.standard_t(2.25, n) 
                # Y
                Y = beta[0] + X.dot(beta[1:]) +  err
                

                ############################################### 
                #-------------  Private (proposed)  ----------- 
                ############################################### 

                #----------- index selection -----------
                xi = np.sqrt(np.log(n)+np.log(p))  
                U = X * Y[:, None]
                absU = np.abs(U)
                weights = np.where(absU > 0.0, np.minimum(xi / absU, 1.0), 1.0)
                U_tilde = U * weights
                g = np.abs(U_tilde.mean(axis=0)) 
                # Global replace-one sensitivity for each score: Δ = 2C / n
                Delta = 2.0 * xi / n
                s = int(np.ceil(1.2 * s_star))
                dx_peel_1 = dp_top_s_laplace_peeling(g, s-1, eps_s[0], Delta, rng=rgt)
                dx_peel_2 = dp_top_s_laplace_peeling(g, s-1, eps_s[1], Delta, rng=rgt)
        
                #----------- initialization -----------
                support_org_1 = np.asarray(list(dx_peel_1), dtype=int)
                support_org_2 = np.asarray(list(dx_peel_2), dtype=int)
                X_low_1 = X[:,support_org_1]
                X_low_2 = X[:,support_org_2]
                epsilon_tau =  eps_init/4  # privacy budget for DP standard deviation estimation: 1/4
                epsilon_ridge =  3*eps_init/4 # privacy budget for DP huber+ridge estimation: 3/4
                tau_init_1 =   St_DP(Y, epsilon = epsilon_tau[0]) 
                tau_init_2 =   St_DP(Y, epsilon = epsilon_tau[1])   
                beta_0DP_1 =  huber_ridge_priv(X_low_1, Y, tau=tau_init_1, epsilon= epsilon_ridge[0], delta= delta_init) 
                beta_0DP_2 =  huber_ridge_priv(X_low_2, Y, tau=tau_init_2, epsilon= epsilon_ridge[1], delta= delta_init)
                beta0DP_1 = np.zeros(p)
                support_selected_1 = np.insert(support_org_1 + 1, 0, 0).astype(int) 
                beta0DP_1[support_selected_1] = beta_0DP_1 
                beta0DP_2 = np.zeros(p)
                support_selected_2 = np.insert(support_org_2 + 1, 0, 0).astype(int) 
                beta0DP_2[support_selected_2] = beta_0DP_2 
                
                #----------- estimation -----------
                tau_priv_1 = .04 * tau_init_1 * (n*epsilon[0]/(s*np.log(p)+np.log(n)))**0.5 # robustification parameter for  DP    
                tau_priv_2 = .04 * tau_init_2 * (n*epsilon[1]/(s*np.log(p)+np.log(n)))**0.5 
                lr_priv =  0.01  
                B_priv = 0.5*(np.log(p) + np.log(n))**0.5
                T_priv = int(np.ceil(2*np.log(n))) # number of iterations
                if T_priv*np.log(T_priv/delta_main) < 2.5*np.log(2/delta_main)*np.log((2*T_priv)/delta_main):
                    epsilon_scale_priv = eps_main/T_priv
                    delta_scale_priv = delta_main/T_priv
                else: 
                    epsilon_scale_priv = eps_main/np.sqrt(2.5*T_priv*np.log(2/delta_main))
                    delta_scale_priv = delta_main/ (2*T_priv)
                model_est = Huber(X , Y ,intercept=True)
                out_priv_1 = model_est.noisygd_highdim( s=s, lr=lr_priv, T=T_priv,  tau=tau_priv_1 , beta0=beta0DP_1,  B_high=B_priv,  epsilon_scale =epsilon_scale_priv[0],  delta_scale=delta_scale_priv )    
                priv_rela_l2_1  = np.log((np.sum((out_priv_1['beta']  - beta )**2 ))**0.5  / beta_norm )
                priv_rela_l2_comp = np.log((np.sum((out_priv_1['beta'][1:]  - beta_slope )**2 ))**0.5  / beta_slope_norm ) # epsilon = 0.5 for comparison with DP-ls
                out_priv_2 = model_est.noisygd_highdim( s=s, lr=lr_priv, T=T_priv,  tau=tau_priv_2 , beta0=beta0DP_2,  B_high=B_priv,  epsilon_scale =epsilon_scale_priv[1],  delta_scale=delta_scale_priv )    
                priv_rela_l2_2  = np.log((np.sum((out_priv_2['beta']  - beta )**2 ))**0.5  / beta_norm )
                    
                
                ############################################### 
                #----------------  Non-private  --------------- 
                # ###############################################  
                tau0 = np.sqrt(np.mean(Y**2)-(np.mean(Y)**2))
                tau_np_high = .1 *tau0  * (n/(s*np.log(p)+np.log(n)))**0.5 # robustification parameter for noiseless Gaussian DP 
                lr_np =  0.2  
                T_np = int(np.ceil(2*np.log(n))) # number of iterations 
                model_est = Huber(X , Y ,intercept=True)
                out_np = model_est.gd_highdim(s=s, lr=lr_np, T=T_np,  tau=tau_np_high, beta0=np.array([]),  standardize= False) 
                rela_l2  =  np.log((np.sum((out_np['beta']   - beta  )**2))**0.5  / beta_norm)
                rela_l2_comp = np.log((np.sum((out_np['beta'][1:]  - beta_slope )**2 ))**0.5  / beta_slope_norm ) # for comparison with DP-ls

                
                re_all = np.array([ priv_rela_l2_1,priv_rela_l2_2,rela_l2, priv_rela_l2_comp,rela_l2_comp])
                results.append(re_all)
    return results

In [8]:
# test  
cuda_cores = 20
M = 300
re_high_priv_est = np.array(Parallel(n_jobs=cuda_cores)(delayed(main_priv_est_parallel)(m ) for m in tqdm(range(M))))

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
# save and reload
np.save("re_high.npy", re_high_priv_est)
#re_high_priv_est = np.load("re_high.npy")

### Differnt privacy levels

#### Plot function

In [9]:
def plot_re_eps(re_eps_p5000, n_set): 
    arr = np.asarray(re_eps_p5000)
    n_set = np.asarray(n_set)

    M, combos, n_methods = arr.shape
    assert combos == 6, "Second dimension of re_eps_p5000 must be 6 (2 noise types × 3 n's)."
    assert n_methods == 3, "Last dimension must be 3 (eps=0.5, eps=0.9, Non-private)."
    assert len(n_set) == 3, "This function currently assumes 3 sample sizes."

    # Reshape to (M, 2 noise types, 3 sample sizes, 3 methods)
    arr4 = arr.reshape(M, 2, 3, 3)

    # Colors and labels for the 3 methods
    method_colors = {
        0: 'orange',      # DP, epsilon = 0.5
        1: 'red',         # DP, epsilon = 0.9
        2: 'deepskyblue'  # Non-private
    }
    method_labels = {
        0: 'epsilon = 0.5',
        1: 'epsilon = 0.9',
        2: 'Non-private'
    }

    num_methods = 3
    num_n = len(n_set)

    # For each sample size n, we place 3 method boxplots side by side,
    # and leave some gap between different n-groups.
    group_width = num_methods + 1.0
    group_centers = np.arange(num_n) * group_width

    noise_names = [r'$N(0,1)\ \mathrm{noise}$', r'$t_{2.25}\ \mathrm{noise}$']

    fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=300)
    axs = np.atleast_1d(axs)

    for noise_idx in range(2):
        ax = axs[noise_idx]

        box_data = []
        box_positions = []

        # Loop over sample sizes and methods
        for k in range(num_n):          # index of n
            for m_idx in range(num_methods):  # index of method
                # Values across M replications
                vals = arr4[:, noise_idx, k, m_idx]  # shape: (M,)
                box_data.append(vals)

                # Horizontal position of this box
                pos = group_centers[k] + m_idx
                box_positions.append(pos)

        # Draw boxplots
        bp = ax.boxplot(
            box_data,
            positions=box_positions,
            widths=0.6,
            patch_artist=True,
            showfliers=False  # hide outliers to avoid extreme compression of y-axis
        )

        # Color the boxes according to method
        for i, patch in enumerate(bp['boxes']):
            m_idx = i % num_methods
            patch.set_facecolor(method_colors[m_idx])
            patch.set_edgecolor('black')
            patch.set_alpha(0.7)

        # Style the medians
        for median in bp['medians']:
            median.set_color('black')
            median.set_linewidth(1.2)

        # Set x-ticks at the center of each group (corresponding to each n)
        ax.set_xticks(group_centers + (num_methods - 1) / 2.0)
        xtick_labels = [rf'$n={int(v)}$' for v in n_set]
        ax.set_xticklabels(xtick_labels, fontsize=10)
        ax.set_xlabel('Sample size', fontsize=11, fontweight='bold')

        # Draw vertical dashed lines between groups of different n
        for k in range(num_n - 1):
            last_k = group_centers[k] + (num_methods - 1)
            first_k1 = group_centers[k+1]
            x_sep = 0.5 * (last_k + first_k1)
            ax.axvline(
                x=x_sep,
                color='grey',
                linestyle='--',
                linewidth=1,
                alpha=0.6,
                zorder=0
            )

        # y-axis label: log relative L2 error
        ax.set_ylabel(
            r'${\rm \mathbf{Log~relative~}}~ \boldsymbol{\ell}_2$-error',
            fontsize=11,
            fontweight='bold'
        )
        ax.set_title(noise_names[noise_idx], fontsize=12, fontweight='bold')

        # Axis styling
        ax.set_facecolor('white')
        ax.spines['left'].set_visible(True)
        ax.spines['bottom'].set_visible(True)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        ax.spines['left'].set_linewidth(1.5)
        ax.spines['bottom'].set_linewidth(1.5)
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
        ax.tick_params(axis='both', which='major', labelsize=9)

    # Shared legend for the three methods
    legend_handles = [
        Line2D([0], [0], color=method_colors[i], lw=3)
        for i in range(num_methods)
    ]
    legend_labels = [method_labels[i] for i in range(num_methods)]
    fig.legend(
        legend_handles,
        legend_labels,
        loc='center left',
        bbox_to_anchor=(0.95, 0.5),
        frameon=True,
        prop={'size': 10}
    )

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    return fig, axs


In [ ]:
re_eps_p5000 = re_high_priv_est[:, 0:6, :3] # different epsilon for p = 5000
re_eps_p10000 = re_high_priv_est[:, 6:12, :3] # different epsilon for p = 10000
n_set = [5000, 10000, 15000]
fig, axs = plot_re_eps(re_eps_p10000, n_set)
#fig.savefig("Est_high_p5000.png", dpi=300, bbox_inches='tight')

### Comparison results

In [ ]:
re_comp_p5000 = re_high_priv_est[:, 0:6, 3:] # comparison results with DP-LS for p = 5000
re_comp_p10000 = re_high_priv_est[:, 6:12, 3:] # comparison results with DP-LS for p = 10000
re_comp_p5000_array = np.round(np.mean(re_comp_p5000, axis=0),3)
re_comp_p10000_array = np.round(np.mean(re_comp_p10000, axis=0),3)
re_comp_p5000_array
#re_comp_p10000_array

In [ ]:
import pandas as pd
df_high_p5000 = pd.DataFrame(re_comp_p5000_array) 
df_high_p5000.to_csv("results_highdim_comp_p5000.csv", index=False)
df_high_p10000 = pd.DataFrame(re_comp_p10000_array) 
df_high_p10000.to_csv("results_highdim_comp_p10000.csv", index=False)